# Set up

In [1]:
import numpy as np
import pandas as pd
import ast
import os
import matplotlib.pyplot as plt

# Data

In [2]:
def extract_data(csv_path, col_name = "estimated_params"):
    df = pd.read_csv(csv_path)
    estimated_params_list = df[col_name].apply(ast.literal_eval).tolist()
    return np.array(estimated_params_list)

In [3]:
X_train = pd.read_csv('Data/X_train.csv', header=None, delimiter=',').values
X_test = pd.read_csv('Data/X_test.csv', header=None, delimiter=',').values

Y_train_21 = pd.read_csv('Data/Y_train_std_21.csv', header=None, delimiter=',').values
Y_test_21 = pd.read_csv('Data/Y_test_std_21.csv', header=None, delimiter=',').values

Y_train_std = pd.read_csv('Data/Y_train_std.csv', header=None, delimiter=',').values
Y_test_std = pd.read_csv('Data/Y_test_std.csv', header=None, delimiter=',').values

In [4]:
L_GP_params = extract_data("PlotData/L.GP_21_result.csv")

L_MGP_params = extract_data("PlotData/L.MGP_21_result.csv")

L_DKMGP_params = extract_data("PlotData/L.DKMGP_21_result.csv")

MVGP_params = extract_data("PlotData/MVGP_21_result.csv")

DNN_params = extract_data("PlotData/DNN_21_result.csv")

# Plots

In [5]:
base_dir = "PlotData"

model_files = {
    "L_GP": "L.GP_21_result.csv",
    "L_MGP": "L.MGP_21_result.csv",
    "L_DKMGP": "L.DKMGP_21_result.csv",
    "MVGP": "MVGP_21_result.csv",
    "DNN": "DNN_21_result.csv"
}

## pre

In [9]:
mse_results = {}
for model, filename in model_files.items():
    est = extract_data(os.path.join(base_dir, filename), col_name = 'test_preds')

    mse_results[model] = np.mean((est - Y_test_21) ** 2)



mse_df = pd.DataFrame.from_dict(mse_results, orient='index', columns=['MSE'])
mse_df.index.name = 'Model'
mse_df = mse_df.reset_index()


print(mse_df)


     Model       MSE
0     L_GP  0.003194
1    L_MGP  0.009780
2  L_DKMGP  0.001409
3     MVGP  0.000643
4      DNN  0.003068


## Single vs. Multi

In [ ]:
mse_dict = {}
for model, filename in model_files.items():
    est = extract_data(os.path.join(base_dir, filename))

    mse_per_row = np.mean((est - X_test) ** 2, axis=1)
    mse_dict[model] = mse_per_row



mse_df = pd.DataFrame(mse_dict)


In [ ]:
plt.figure(figsize=(8, 6))
plt.boxplot([mse_df[model] for model in mse_df.columns], labels=mse_df.columns)
plt.ylabel("Mean Squared Error per Sample")
plt.title("Model Comparison")
plt.grid(True, linestyle='--', linewidth=0.5)
# plt.ylim(0, 0.5)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
ax.boxplot([mse_df[model] for model in mse_df.columns], labels=mse_df.columns)
ax.set_yscale('log')
ax.set_ylabel("MSE per Sample (log scale)")
ax.set_title("Comparison with Log-scaled Y-axis")
ax.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()